In [1]:
%%capture
from pprint import pprint

In [ ]:
from LCBench.api import Benchmark
import os

os.makedirs("LCBench/cached", exist_ok=True)
bench_dir = "LCBench/cached/six_datasets_lw.json"
bench = Benchmark(bench_dir, cache=False)

==> Loading data...
==> No cached data found or cache set to False.
==> Reading json data...
==> Done.


In [3]:
import torch

# use a GPU if available
torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Set default tensor type to float64
torch.set_default_dtype(torch.float64)

In [4]:
def normalize_config(config):
    # Convert each value to a torch tensor (ensuring float type for calculations)
    batch = torch.tensor(config["batch_size"])
    lr = torch.tensor(config["learning_rate"])
    units = torch.tensor(config["max_units"])
    momentum = torch.tensor(config["momentum"])
    weight_decay = torch.tensor(config["weight_decay"])
    layers = torch.tensor(float(config["num_layers"]))
    dropout = torch.tensor(config["max_dropout"])
    
    # For log-scaled parameters: batch size, learning rate, and max units.
    batch_norm = (torch.log(batch) - torch.log(torch.tensor(16.0))) / (torch.log(torch.tensor(512.0)) - torch.log(torch.tensor(16.0)))
    lr_norm = (torch.log(lr) - torch.log(torch.tensor(1e-4))) / (torch.log(torch.tensor(1e-1)) - torch.log(torch.tensor(1e-4)))
    units_norm = (torch.log(units) - torch.log(torch.tensor(64.0))) / (torch.log(torch.tensor(1024.0)) - torch.log(torch.tensor(64.0)))
    
    # For linearly scaled parameters.
    momentum_norm = (momentum - 0.1) / (0.99 - 0.1)
    weight_decay_norm = (weight_decay - 1e-5) / (1e-1 - 1e-5)
    layers_norm = (layers - 1) / (4 - 1)
    
    # Dropout is already between 0 and 1.
    dropout_norm = dropout

    # Combine into a 7-dimensional tensor.
    normalized_vector = torch.stack([
        batch_norm, 
        lr_norm, 
        momentum_norm, 
        weight_decay_norm, 
        layers_norm, 
        units_norm, 
        dropout_norm
    ])
    
    return normalized_vector

In [5]:
all_x = []
all_y = []
all_c = []
dataset_name = "higgs"
for config_id in bench.data[dataset_name].keys():
    config = bench.query(dataset_name, "config", config_id)
    all_x.append(normalize_config(config))
    val_ce = bench.query(dataset_name, "final_val_cross_entropy", config_id)
    all_y.append(val_ce)
    runtime = bench.query(dataset_name, "time", config_id)[-1]
    all_c.append(runtime)

all_x = torch.stack(all_x)
all_y = torch.tensor(all_y).unsqueeze(1)
all_c = torch.tensor(all_c).unsqueeze(1)

In [6]:
all_x, all_y, all_c

(tensor([[0.4299, 0.4204, 0.1272,  ..., 0.6667, 0.5487, 0.0259],
         [0.9672, 0.6977, 0.0720,  ..., 1.0000, 0.9729, 0.5472],
         [0.8919, 0.1077, 0.3272,  ..., 0.0000, 0.8208, 0.3320],
         ...,
         [0.6750, 0.8598, 0.4454,  ..., 0.6667, 0.4707, 0.3635],
         [0.9691, 0.3290, 0.0093,  ..., 0.3333, 0.8684, 0.0437],
         [0.3666, 0.9906, 0.2041,  ..., 1.0000, 0.6681, 0.4045]]),
 tensor([[0.6380],
         [0.6931],
         [0.7014],
         ...,
         [0.6090],
         [0.6511],
         [0.6931]]),
 tensor([[215.9746],
         [876.6520],
         [126.5395],
         ...,
         [186.1553],
         [165.5783],
         [877.6618]]))

In [7]:
from pandora_automl.utils import fit_gp_model

In [8]:
from botorch.acquisition import LogExpectedImprovement

In [9]:
n_iter = 3
maximize = False  # We want to minimize final_val_cross_entropy

torch.manual_seed(42)
init_config_id = torch.randint(low=0, high=2000, size=(1,)).item()
print(f"  Initial config_id: {init_config_id}")
config_id_history = [init_config_id]
x = all_x[init_config_id].unsqueeze(0)
y = all_y[init_config_id].unsqueeze(0)

for iteration in range(n_iter):
    # 1. Fit a GP model on the current data.
    model = fit_gp_model(x, y, output_standardize=True)
    
    # 2. Determine the best observed objective value.
    best_f = y.min()
    
    # 3. Define the acquisition function.
    LogEI = LogExpectedImprovement(model=model, best_f=best_f, maximize=maximize)
    
    # 4. Evaluate the acquisition function on all candidate x's.
    # The unsqueeze operations add extra dimensions if required by your model.
    acq = LogEI.forward(all_x.unsqueeze(1))
    
    # 5. Select the candidate with the highest acquisition value.
    new_config_id = torch.argmax(acq)
    config_id_history.append(new_config_id.item())
    new_config_x = all_x[new_config_id]
    new_config_LogEI = torch.max(acq)
    
    # 6. Query the objective for the new configuration.
    new_config_y = all_y[new_config_id]
    
    # 7. Append the new data to our training set.
    x = torch.cat([x, new_config_x.unsqueeze(0)], dim=0)
    y = torch.cat([y, new_config_y.unsqueeze(0)], dim=0)
    
    print(f"Iteration {iteration + 1}:")
    print(f"  Selected config_id: {new_config_id}")
    print(f"  Acquisition (LogEI): {new_config_LogEI.item():.4f}")
    print(f"  Objective (final_val_cross_entropy): {new_config_y.item():.4f}")
    print(f"  Current best observed: {y.min().item():.4f}")
    print()

  Initial config_id: 1542


/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/transforms/outcome.py:304: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  stdvs = Y.std(dim=-2, keepdim=True)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)


Iteration 1:
  Selected config_id: 1
  Acquisition (LogEI): -0.3169
  Objective (final_val_cross_entropy): 0.6931
  Current best observed: 0.6422

Iteration 2:
  Selected config_id: 1761
  Acquisition (LogEI): -4.5126
  Objective (final_val_cross_entropy): 0.6420
  Current best observed: 0.6420

Iteration 3:
  Selected config_id: 504
  Acquisition (LogEI): -4.8528
  Objective (final_val_cross_entropy): 0.6455
  Current best observed: 0.6420



In [10]:
from pandora_automl.acquisition.gittins import GittinsIndex

In [11]:
n_iter = 3
maximize = False  # We want to minimize final_val_cross_entropy
lmbda = 0.003

torch.manual_seed(42)
init_config_id = torch.randint(low=0, high=2000, size=(1,)).item()
print(f"  Initial config_id: {init_config_id}")
config_id_history = [init_config_id]
x = all_x[init_config_id].unsqueeze(0)
y = all_y[init_config_id].unsqueeze(0)
acq_history = []

for iteration in range(n_iter):
    # 1. Fit a GP model on the current data.
    model = fit_gp_model(x, y, output_standardize=True)
    
    # 3. Define the acquisition function.
    PBGI = GittinsIndex(model=model, maximize=maximize, lmbda=lmbda)
    
    # 4. Evaluate the acquisition function on all candidate x's.
    # The unsqueeze operations add extra dimensions if required by your model.
    acq = PBGI.forward(all_x.unsqueeze(1))
    
    # 5. Select the candidate with the lowest acquisition value.
    new_config_id = torch.argmin(acq)
    new_config_x = all_x[new_config_id]
    new_config_GI = torch.min(acq)
    
    # 6. Query the objective for the new configuration.
    new_config_y = all_y[new_config_id]
    
    # 7. Append the new data to our training set.
    x = torch.cat([x, new_config_x.unsqueeze(0)], dim=0)
    y = torch.cat([y, new_config_y.unsqueeze(0)], dim=0)
    config_id_history.append(new_config_id.item())
    acq_history.append(new_config_GI.item())
    
    print(f"Iteration {iteration + 1}:")
    print(f"  Selected config_id: {new_config_id}")
    print(f"  Acquisition (GI): {new_config_GI.item():.4f}")
    print(f"  Objective (final_val_cross_entropy): {new_config_y.item():.4f}")
    print(f"  Current best observed: {y.min().item():.4f}")
    print()

  Initial config_id: 1542
Iteration 1:
  Selected config_id: 1
  Acquisition (GI): -4.0382
  Objective (final_val_cross_entropy): 0.6931
  Current best observed: 0.6422

Iteration 2:
  Selected config_id: 1761
  Acquisition (GI): 0.6076
  Objective (final_val_cross_entropy): 0.6420
  Current best observed: 0.6420

Iteration 3:
  Selected config_id: 1981
  Acquisition (GI): 0.6248
  Objective (final_val_cross_entropy): 0.6419
  Current best observed: 0.6419



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/transforms/outcome.py:304: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  stdvs = Y.std(dim=-2, keepdim=True)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)


In [18]:
from botorch.acquisition import UpperConfidenceBound
from pandora_automl.acquisition.lcb import LowerConfidenceBound

In [20]:
import numpy as np
import math
n_iter = 3
dim = 7
maximize = False  # We want to minimize final_val_cross_entropy

torch.manual_seed(42)
init_config_id = torch.randint(low=0, high=2000, size=(1,)).item()
print(f"  Initial config_id: {init_config_id}")
config_id_history = [init_config_id]
x = all_x[init_config_id].unsqueeze(0)
y = all_y[init_config_id].unsqueeze(0)

for iteration in range(n_iter):
    # 1. Fit a GP model on the current data.
    model = fit_gp_model(x, y, output_standardize=True)
    
    # 3. Define the acquisition function.
    LCB = LowerConfidenceBound(model=model, maximize=maximize, beta=2 * np.log(dim * ((iteration + 1) ** 2) * (math.pi ** 2) / (6 * 0.1)) / 5)
    
    # 4. Evaluate the acquisition function on all candidate x's.
    # The unsqueeze operations add extra dimensions if required by your model.
    acq = LCB.forward(all_x.unsqueeze(1))
    
    # 5. Select the candidate with the lowest acquisition value.
    new_config_id = torch.argmin(acq)
    config_id_history.append(new_config_id.item())
    new_config_x = all_x[new_config_id]
    new_config_LCB = torch.min(acq)

    UCB = UpperConfidenceBound(model=model, maximize=maximize, beta=2 * np.log(dim * ((iteration + 1) ** 2) * (math.pi ** 2) / (6 * 0.1)) / 5)
    min_UCB = torch.min(UCB.forward(all_x.unsqueeze(1)))
    regret_upper_bound = min_UCB - new_config_LCB
    
    # 6. Query the objective for the new configuration.
    new_config_y = all_y[new_config_id]
    
    # 7. Append the new data to our training set.
    x = torch.cat([x, new_config_x.unsqueeze(0)], dim=0)
    y = torch.cat([y, new_config_y.unsqueeze(0)], dim=0)
    
    print(f"Iteration {iteration + 1}:")
    print(f"  Selected config_id: {new_config_id}")
    print(f"  Acquisition (LCB): {new_config_LCB.item():.4f}")
    print(f"  Regret Upper Bound: {regret_upper_bound.item():.4f}")
    print(f"  Objective (final_val_cross_entropy): {new_config_y.item():.4f}")
    print()

  Initial config_id: 1542
Iteration 1:
  Selected config_id: 1
  Acquisition (LCB): -3.1579
  Regret Upper Bound: 2.5294
  Objective (final_val_cross_entropy): 0.6931

Iteration 2:
  Selected config_id: 131
  Acquisition (LCB): -0.7480
  Regret Upper Bound: 0.0554
  Objective (final_val_cross_entropy): 0.6921

Iteration 3:
  Selected config_id: 996
  Acquisition (LCB): -0.7325
  Regret Upper Bound: 0.0399
  Objective (final_val_cross_entropy): 0.6931



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/transforms/outcome.py:304: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  stdvs = Y.std(dim=-2, keepdim=True)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)


In [14]:
from pandora_automl.acquisition.log_ei_puc import LogExpectedImprovementWithCost

In [15]:
n_iter = 3
maximize = False  # We want to minimize final_val_cross_entropy

torch.manual_seed(42)
init_config_id = torch.randint(low=0, high=2000, size=(1,)).item()
print(f"  Initial config_id: {init_config_id}")
config_id_history = [init_config_id]
x = all_x[init_config_id].unsqueeze(0)
y = all_y[init_config_id].unsqueeze(0)
c = all_c[init_config_id].unsqueeze(0)

for iteration in range(n_iter):
    # 1. Fit a GP model on the current data.
    model = fit_gp_model(X=x, objective_X=y, cost_X=c, unknown_cost=True, output_standardize=True)
    
    # 2. Determine the best observed objective value.
    best_f = y.min()
    
    # 3. Define the acquisition function.
    LogEIC = LogExpectedImprovementWithCost(model=model, best_f=best_f, maximize=maximize, unknown_cost=True)
    
    # 4. Evaluate the acquisition function on all candidate x's.
    # The unsqueeze operations add extra dimensions if required by your model.
    acq = LogEIC.forward(all_x.unsqueeze(1))
    
    # 5. Select the candidate with the highest acquisition value.
    new_config_id = torch.argmax(acq)
    config_id_history.append(new_config_id.item())
    new_config_x = all_x[new_config_id]
    new_config_LogEIC = torch.max(acq)
    
    # 6. Query the objective for the new configuration.
    new_config_y = all_y[new_config_id]
    new_config_c = all_c[new_config_id]
    
    # 7. Append the new data to our training set.
    x = torch.cat([x, new_config_x.unsqueeze(0)], dim=0)
    y = torch.cat([y, new_config_y.unsqueeze(0)], dim=0)
    c = torch.cat([c, new_config_c.unsqueeze(0)], dim=0)
    
    print(f"Iteration {iteration + 1}:")
    print(f"  Selected config_id: {new_config_id}")
    print(f"  Acquisition (LogEIC): {new_config_LogEIC.item():.6f}")
    print(f"  Objective (final_val_cross_entropy): {new_config_y.item():.6f}")
    print(f"  Cost (final_val_cross_entropy): {new_config_c.item():.6f}")
    print(f"  Current best observed: {y.min().item():.6f}")
    print()

  Initial config_id: 1542
Iteration 1:
  Selected config_id: 1
  Acquisition (LogEIC): -3.818207
  Objective (final_val_cross_entropy): 0.693145
  Cost (final_val_cross_entropy): 876.651991
  Current best observed: 0.642218

Iteration 2:
  Selected config_id: 1761
  Acquisition (LogEIC): -9.168875
  Objective (final_val_cross_entropy): 0.641996
  Cost (final_val_cross_entropy): 251.388938
  Current best observed: 0.641996

Iteration 3:
  Selected config_id: 504
  Acquisition (LogEIC): -10.126739
  Objective (final_val_cross_entropy): 0.645500
  Cost (final_val_cross_entropy): 199.601905
  Current best observed: 0.641996



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/transforms/outcome.py:304: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  stdvs = Y.std(dim=-2, keepdim=True)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)


In [16]:
lmbda = 0.00001
n_iter = 3

torch.manual_seed(42)
init_config_id = torch.randint(low=0, high=2000, size=(1,)).item()
print(f"  Initial config_id: {init_config_id}")
config_id_history = [init_config_id]
x = all_x[init_config_id].unsqueeze(0)
y = all_y[init_config_id].unsqueeze(0)
c = all_c[init_config_id].unsqueeze(0)
acq_history = []

for iteration in range(n_iter):
    # 1. Fit a GP model on the current data.
    model = fit_gp_model(X=x, objective_X=y, cost_X=c, unknown_cost=True, output_standardize=True)
    
    # 3. Define the acquisition function.
    PBGI = GittinsIndex(model=model, maximize=maximize, lmbda=lmbda, unknown_cost=True)
    
    # 4. Evaluate the acquisition function on all candidate x's.
    # The unsqueeze operations add extra dimensions if required by your model.
    acq = PBGI.forward(all_x.unsqueeze(1))
    
    # 5. Select the candidate with the lowest acquisition value.
    new_config_id = torch.argmin(acq)
    new_config_x = all_x[new_config_id]
    new_config_GI = torch.min(acq)
    
    # 6. Query the objective for the new configuration.
    new_config_y = all_y[new_config_id]
    new_config_c = all_c[new_config_id]
    
    # 7. Append the new data to our training set.
    x = torch.cat([x, new_config_x.unsqueeze(0)], dim=0)
    y = torch.cat([y, new_config_y.unsqueeze(0)], dim=0)
    c = torch.cat([c, new_config_c.unsqueeze(0)], dim=0)
    config_id_history.append(new_config_id.item())
    acq_history.append(new_config_GI.item())
    
    print(f"Iteration {iteration + 1}:")
    print(f"  Selected config_id: {new_config_id}")
    print(f"  Acquisition (GI): {new_config_GI.item():.4f}")
    print(f"  Objective (final_val_cross_entropy): {new_config_y.item():.4f}")
    print(f"  Cost (final_val_cross_entropy): {new_config_c.item():.4f}")
    print(f"  Current best observed: {y.min().item():.4f}")
    print()

  Initial config_id: 1542
Iteration 1:
  Selected config_id: 1
  Acquisition (GI): -3.3481
  Objective (final_val_cross_entropy): 0.6931
  Cost (final_val_cross_entropy): 876.6520
  Current best observed: 0.6422



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/transforms/outcome.py:304: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  stdvs = Y.std(dim=-2, keepdim=True)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)


Iteration 2:
  Selected config_id: 1761
  Acquisition (GI): 0.6360
  Objective (final_val_cross_entropy): 0.6420
  Cost (final_val_cross_entropy): 251.3889
  Current best observed: 0.6420

Iteration 3:
  Selected config_id: 504
  Acquisition (GI): 0.6294
  Objective (final_val_cross_entropy): 0.6455
  Cost (final_val_cross_entropy): 199.6019
  Current best observed: 0.6420



In [ ]:
lmbda = 0.00001
n_iter = 3
maximize = False

torch.manual_seed(42)
init_config_id = torch.randint(low=0, high=2000, size=(1,)).item()
print(f"  Initial config_id: {init_config_id}")
config_id_history = [init_config_id]
x = all_x[init_config_id].unsqueeze(0)
y = all_y[init_config_id].unsqueeze(0)
c = all_c[init_config_id].unsqueeze(0)
acq_history = []
stp_history = []

for iteration in range(n_iter):
    # 1. Fit a GP model on the current data.
    model = fit_gp_model(X=x, objective_X=y, cost_X=c, unknown_cost=True, output_standardize=True)
    
    # 2. Determine the best observed objective value.
    best_f = y.min()
    
    # 3. Define the acquisition function.
    PBGI = GittinsIndex(model=model, maximize=maximize, lmbda=lmbda, unknown_cost=True)
    LogEIC = LogExpectedImprovementWithCost(model=model, best_f=best_f, maximize=maximize, unknown_cost=True)
    
    # 4. Evaluate the acquisition function on all candidate x's.
    # The unsqueeze operations add extra dimensions if required by your model.
    acq = PBGI.forward(all_x.unsqueeze(1))
    stp = LogEIC.forward(all_x.unsqueeze(1))
    
    # 5. Select the candidate with the lowest acquisition value.
    new_config_id = torch.argmin(acq)
    new_config_x = all_x[new_config_id]
    new_config_GI = torch.min(acq)

    max_config_logEIC = torch.max(stp)
    
    # 6. Query the objective for the new configuration.
    new_config_y = all_y[new_config_id]
    new_config_c = all_c[new_config_id]
    
    # 7. Append the new data to our training set.
    x = torch.cat([x, new_config_x.unsqueeze(0)], dim=0)
    y = torch.cat([y, new_config_y.unsqueeze(0)], dim=0)
    c = torch.cat([c, new_config_c.unsqueeze(0)], dim=0)
    config_id_history.append(new_config_id.item())
    acq_history.append(new_config_GI.item())
    stp_history.append(max_config_logEIC.item())
    
    print(f"Iteration {iteration + 1}:")
    print(f"  Selected config_id: {new_config_id}")
    print(f"  Acquisition (GI): {new_config_GI.item():.4f}")
    print(f"  Stopping acquisition (LogEIC): {max_config_logEIC.item():.4f}")
    print(f"  Objective (final_val_cross_entropy): {new_config_y.item():.4f}")
    print(f"  Cost (final_val_cross_entropy): {new_config_c.item():.4f}")
    print(f"  Current best observed: {y.min().item():.4f}")
    print()

  Initial config_id: 1542
Iteration 1:
  Selected config_id: 1
  Acquisition (GI): -3.3481
  Stopping acquisition (LogEIC): -3.8182
  Objective (final_val_cross_entropy): 0.6931
  Cost (final_val_cross_entropy): 876.6520
  Current best observed: 0.6422



/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/transforms/outcome.py:304: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  stdvs = Y.std(dim=-2, keepdim=True)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)


Iteration 2:
  Selected config_id: 1761
  Acquisition (GI): 0.6360
  Stopping acquisition (LogEIC): -9.1689
  Objective (final_val_cross_entropy): 0.6420
  Cost (final_val_cross_entropy): 251.3889
  Current best observed: 0.6420

Iteration 3:
  Selected config_id: 504
  Acquisition (GI): 0.6294
  Stopping acquisition (LogEIC): -10.1267
  Objective (final_val_cross_entropy): 0.6455
  Cost (final_val_cross_entropy): 199.6019
  Current best observed: 0.6420



In [ ]:
n_iter = 3
maximize = False
acq = "PBGI_D"

torch.manual_seed(42)
init_config_id = torch.randint(low=0, high=2000, size=(1,)).item()
print(f"  Initial config_id: {init_config_id}")
config_id_history = [init_config_id]
x = all_x[init_config_id].unsqueeze(0)
y = all_y[init_config_id].unsqueeze(0)
c = all_c[init_config_id].unsqueeze(0)
PBGI_1e_5_acq_history = [np.nan]
PBGI_5e_6_acq_history = [np.nan]
PBGI_1e_6_acq_history = [np.nan]
LogEIC_acq_history = [np.nan]
regret_upper_bound_history = [np.nan]
acq_history = [np.nan]
current_best_history = []
cost_history = [0]
if acq == "PBGI_D":
    cur_lmbda = 0.001
    lmbda_history = [cur_lmbda]

for iteration in range(n_iter):
    # 1. Fit a GP model on the current data.
    model = fit_gp_model(X=x, objective_X=y, cost_X=c, unknown_cost=True, output_standardize=True)
    
    # 2. Determine the best observed objective value.
    best_f = y.min()
    
    # 3. Define the acquisition function.
    PBGI_1e_5 = GittinsIndex(model=model, maximize=maximize, lmbda=1e-5, unknown_cost=True)
    PBGI_5e_6 = GittinsIndex(model=model, maximize=maximize, lmbda=5e-6, unknown_cost=True)
    PBGI_1e_6 = GittinsIndex(model=model, maximize=maximize, lmbda=1e-6, unknown_cost=True)
    LogEIC = LogExpectedImprovementWithCost(model=model, best_f=best_f, maximize=maximize, unknown_cost=True)
    single_outcome_model = fit_gp_model(X=x, objective_X=y, output_standardize=True)
    UCB = UpperConfidenceBound(model=single_outcome_model, maximize=maximize, beta=2 * np.log(dim * ((iteration + 1) ** 2) * (math.pi ** 2) / (6 * 0.1)) / 5)
    LCB = LowerConfidenceBound(model=single_outcome_model, maximize=maximize, beta=2 * np.log(dim * ((iteration + 1) ** 2) * (math.pi ** 2) / (6 * 0.1)) / 5)

    # 4. Evaluate the acquisition function on all candidate x's.
    # The unsqueeze operations add extra dimensions if required by your model.
    PBGI_1e_5_acq = PBGI_1e_5.forward(all_x.unsqueeze(1))
    PBGI_5e_6_acq = PBGI_5e_6.forward(all_x.unsqueeze(1))
    PBGI_1e_6_acq = PBGI_1e_6.forward(all_x.unsqueeze(1))
    LogEIC_acq = LogEIC.forward(all_x.unsqueeze(1))
    UCB_acq = UCB.forward(all_x.unsqueeze(1))
    LCB_acq = LCB.forward(all_x.unsqueeze(1))

    # 5. Record information for stopping.
    PBGI_1e_5_acq_history.append(torch.min(PBGI_1e_5_acq).item())
    PBGI_5e_6_acq_history.append(torch.min(PBGI_5e_6_acq).item())
    PBGI_1e_6_acq_history.append(torch.min(PBGI_1e_6_acq).item())
    LogEIC_acq_history.append(torch.max(LogEIC_acq).item())
    regret_upper_bound_history.append(torch.min(UCB_acq).item() - torch.min(LCB_acq).item())

    # 6. Select the candidate with the lowest acquisition value.
    if acq == "PBGI(1e-5)":
        new_config_id = torch.argmin(PBGI_1e_5_acq)
        new_config_acq = torch.min(PBGI_1e_5_acq)
    if acq == "PBGI(5e-6)":
        new_config_id = torch.argmin(PBGI_5e_6_acq)
        new_config_acq = torch.min(PBGI_5e_6_acq)
    if acq == "PBGI(1e-6)":
        new_config_id = torch.argmin(PBGI_1e_6_acq)
        new_config_acq = torch.min(PBGI_1e_6_acq)
    if acq == "LogEIC":
        new_config_id = torch.argmax(LogEIC_acq)
        new_config_acq = torch.max(LogEIC_acq)
    if acq == "LCB":
        new_config_id = torch.argmin(LCB_acq)
        new_config_acq = torch.min(LCB_acq)
    if acq == "PBGI_D":
        PBGI_D = GittinsIndex(model=model, maximize=maximize, lmbda=cur_lmbda, unknown_cost=True)
        PBGI_D_acq = PBGI_D.forward(all_x.unsqueeze(1))
        new_config_id = torch.argmin(PBGI_D_acq)
        new_config_acq = torch.min(PBGI_D_acq)
        if new_config_acq > best_f:
            cur_lmbda = cur_lmbda / 2
        lmbda_history.append(cur_lmbda)

    new_config_x = all_x[new_config_id]
    
    # 7. Query the objective for the new configuration.
    new_config_y = all_y[new_config_id]
    new_config_c = all_c[new_config_id]
    
    # 8. Append the new data to our training set.
    x = torch.cat([x, new_config_x.unsqueeze(0)], dim=0)
    y = torch.cat([y, new_config_y.unsqueeze(0)], dim=0)
    c = torch.cat([c, new_config_c.unsqueeze(0)], dim=0)
    config_id_history.append(new_config_id.item())
    acq_history.append(new_config_acq.item())
    current_best_history.append(best_f.item())
    cost_history.append(new_config_c.item())

current_best_history.append(y.min().item())

  Initial config_id: 1542


/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/transforms/outcome.py:304: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  stdvs = Y.std(dim=-2, keepdim=True)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-packages/botorch/models/utils/assorted.py:194: UserWarning: std(): degrees of freedom is <= 0. Correction should be strictly less than the reduction factor (input numel divided by output numel). (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/ReduceOps.cpp:1831.)
  Ymean, Ystd = torch.mean(Y, dim=-2), torch.std(Y, dim=-2)
/Users/qianxie/Desktop/Curriculum/Graduate/Research/PandoraAutoML/automl_env/lib/python3.9/site-

In [33]:
PBGI_1e_5_acq_history, PBGI_5e_6_acq_history, PBGI_1e_6_acq_history, LogEIC_acq_history, regret_upper_bound_history, acq_history, current_best_history, cost_history, lmbda_history

([nan, -3.3481389956653276, -3.3481203721063633, 0.6439566720863261],
 [nan, -3.779742667286553, -3.779720071832438, 0.6429633306136342],
 [nan, -4.685279849694181, -4.685249169382612, 0.641642423499978],
 [nan, -3.818206536171723, -3.8182389923714624, -13.25667469120075],
 [nan, 2.5294199968377984, 2.8705814603731907, 0.0002649271222241456],
 [nan, 0.8177866418706006, -0.06128220687253405, 0.7299982825504507],
 [0.6422176830029588,
  0.6422176830029588,
  0.6419958293894987,
  0.6419958293894987],
 [0, 175.56018090248108, 251.38893842697144, 175.56018090248108],
 [0.001, 0.0005, 0.0005, 0.00025])

In [34]:
np.cumsum(cost_history)

array([  0.        , 175.5601809 , 426.94911933, 602.50930023])

In [35]:
for cost, best, acq in zip(np.cumsum(cost_history), current_best_history, acq_history):
    print(cost, best, acq)

0.0 0.6422176830029588 nan
175.56018090248108 0.6422176830029588 0.8177866418706006
426.9491193294525 0.6419958293894987 -0.06128220687253405
602.5093002319336 0.6419958293894987 0.7299982825504507
